In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets

In [2]:
# load breast cancer dataset
data = datasets.load_breast_cancer()

# convert to pandas data frame
features = data.feature_names
dataDF = pd.DataFrame(data.data, columns=features)

# add binary target variable
target = 'target'
dataDF['target'] = data.target

# display 
dataDF.sample(5)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
148,14.44,15.18,93.97,640.1,0.09970,0.10210,0.08487,0.05532,0.1724,0.06081,...,19.85,108.60,766.9,0.13160,0.27350,0.31030,0.15990,0.2691,0.07683,1
296,10.91,12.35,69.14,363.7,0.08518,0.04721,0.01236,0.01369,0.1449,0.06031,...,14.82,72.42,392.2,0.09312,0.07506,0.02884,0.03194,0.2143,0.06643,1
374,13.69,16.07,87.84,579.1,0.08302,0.06374,0.02556,0.02031,0.1872,0.05669,...,20.21,99.16,670.6,0.11050,0.20960,0.13460,0.06987,0.3323,0.07701,1
326,14.11,12.88,90.03,616.5,0.09309,0.05306,0.01765,0.02733,0.1373,0.05700,...,18.00,98.40,749.9,0.12810,0.11090,0.05307,0.05890,0.2100,0.07083,1
437,14.04,15.98,89.78,611.2,0.08458,0.05895,0.03534,0.02944,0.1714,0.05898,...,21.58,101.20,750.0,0.11950,0.12520,0.11170,0.07453,0.2725,0.07234,1


In [3]:
# split data into train, validation and test
n = dataDF.shape[0]
sizes = [int(0.8 * n), int(0.9 * n)]
trainDF, validationDF, testDF = np.split(dataDF.sample(frac=1, random_state=10), sizes)

print("Train: ", trainDF.shape)
print("Validation: ", validationDF.shape)
print("Test: ", testDF.shape)



Train:  (455, 31)
Validation:  (57, 31)
Test:  (57, 31)


# Simple Logistic Regression

In [4]:
from sklearn.linear_model import LogisticRegression

In [5]:
model = LogisticRegression(solver='lbfgs', max_iter=5000)
model.fit(trainDF[features], trainDF[target])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
predicted = model.predict(testDF[features])

In [7]:
from sklearn.metrics import accuracy_score
print(accuracy_score(testDF[target], predicted))

0.9122807017543859


# Using Adaboost

The idea of adaboost, acronym for "Adaptive Boosting", is simple. Imagine back your school days preparing for a exam. Once you have scanned through and praticsed the whole syllabalus, you would spend more time praticising hard problems. Boosting is based on the same principle. Once we have our initial classifier that treats each data point equally, we start focusing more on data points for which our previous classifier was wrong. The idea here is that the direction of gradient descent is more influenced by data points for which our model having trouble dealing with. One simple way to achieve this is by increasing the weight of the sample. Recall, hwo in chapter [4](04-linearregression.html), we used mean square error (MSE) as our objective function to minimize and it is computed as follows:

$$MSE = \frac{1}{m} \sum_{i=1}^{m}\left[y - \sum_{i=1}^{n}\theta_iX_i\right]^2$$

We can generalize, the above function to incorporate sample weight, say $w_i$, as:

$$MSE = \frac{1}{m} \sum_{i=1}^{m}\left[w_i\left(y - \sum_{i=1}^{n}\theta_iX_i\right)\right]^2$$

Now, if we take partial derivative of the above equation, we get direction of gradient descent that is influenced by the weight of the sample. Thus, we can assign higher weight to difficult data points and influence the direction of the gradient descent towards these difficult data points.

The idea make sense but the problem remains is how to find optimal weight for difficult data points. If it's too low then the gradient descent direction might not change at all and if it's too hight then the gradient descent might gets too much influenced by hard data points and start having trouble with other data points that our model previously able to deal with. Adaboost solves this problem. 

There is still one challenge, 


* AdaBoost --> adaptive boosting

**References**:
1. [Boosting with Adaboost and Gradient Boosting](https://medium.com/diogo-menezes-borges/boosting-with-adaboost-and-gradient-boosting-9cbab2a1af81)
2. [A comprehensive guide to ensemble learning](https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-for-ensemble-models/)

In [8]:
sample_weights = np.repeat(1./trainDF.shape[0], trainDF.shape[0])
model1 = LogisticRegression(solver='lbfgs', max_iter=5000)
model1.fit(trainDF[features], trainDF[target])
predicted = model1.predict(trainDF[features])
print(accuracy_score(trainDF[target], predicted))

0.9626373626373627
